In [1]:
!pip3 install -qU transformers simpletransformers

In [2]:
import numpy as np
import pandas as pd

from nltk.tokenize import sent_tokenize, word_tokenize

from tqdm.notebook import tqdm

In [3]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data = pd.read_csv('drive/MyDrive/DATASETS/MachineLearning'
                   '/topicmodeling/lenta-ru-filtered.csv')

In [6]:
sents = []
for text in tqdm(data['text'].values[:1000]):
    for sent in sent_tokenize(text, language='russian'):
        sents.append(sent)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [7]:
len(sents)

9536

In [8]:
del data

Задана последовательность токенов: $\mathbf{x} = x_1, x_2, \ldots, x_n$

Требуется предсказать последовательность классов: $\mathbf{y} = y_1, y_2, \ldots, y_{n'}$

Где $\mathbf{y}$ востановленная посследовательность токенов с пунктуацией

In [9]:
import re
remove_punk = re.compile(r',')
remove_spaces = re.compile(r'\s+')

sents_train = sents[:4000]
sents_val = sents[4000:]

train_data = []
for sent in tqdm(sents_train):
    try:
        bad_sent = remove_spaces.sub(' ', remove_punk.sub(' ', sent)).lower()
        train_data.append(['correction', bad_sent, sent])
    except:
        continue

val_data = []
for sent in tqdm(sents_val):
    try:
        bad_sent = remove_spaces.sub(' ', remove_punk.sub(' ', sent)).lower()
        val_data.append(['correction', bad_sent, sent])
    except:
        continue

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/5536 [00:01<?, ?it/s]

In [10]:
train_df = pd.DataFrame(train_data, columns=['prefix', 'input_text', 'target_text'])
val_df = pd.DataFrame(val_data, columns=['prefix', 'input_text', 'target_text'])

In [11]:
train_df

,prefix,input_text,target_text
0,correction,с 1 сентября на всей территории россии вводитс...,С 1 сентября на всей территории России вводитс...
1,correction,как пояснили в государственном таможенном коми...,Как пояснили в Государственном таможенном коми...
2,correction,по данным итар-тасс в новой декларации содержи...,"По данным ИТАР-ТАСС, в новой декларации содерж..."
3,correction,так впервые в их число включены ядовитые и сил...,"Так, впервые в их число включены ядовитые и си..."
4,correction,кроме того в декларации будет необходимо приво...,"Кроме того, в декларации будет необходимо прив..."
...,...,...,...
3995,correction,завтра с 10.30 до 11.30 в здании российского ф...,Завтра с 10.30 до 11.30 в здании Российского ф...
3996,correction,ее организацию взяло насебя управление делами ...,Ее организацию взяло насебя Управление делами ...
3997,correction,завтра же днемсостоятся похороны раисы горбаче...,Завтра же днемсостоятся похороны Раисы Горбаче...
3998,correction,54-летний мехрад карими нассери до настоящего ...,"54-летний Мехрад Карими Нассери, до настоящего..."


# Поиграемся с mT5 моделью

In [12]:
from simpletransformers.t5 import T5Model, T5Args

In [13]:
args = T5Args()

args.max_length = 120
args.max_seq_length = 128
args.num_train_epochs = 5
args.train_batch_size = 8
args.use_multiprocessing = True
args.no_save = False
args.evaluate_generated_text = False
args.evaluate_during_training = False
args.evaluate_during_training_verbose = False
args.fp16 = False

In [14]:
model = T5Model('mt5', 'google/mt5-small', args=args, use_cuda=True)

In [15]:
model.train_model(train_df, eval_data=val_df)

  0%|          | 0/4000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3516: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/500 [00:00<?, ?it/s]

Running Epoch 1 of 5:   0%|          | 0/500 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [29]:
model.predict([f'correction: {train_df.loc[1]["input_text"]}'])

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/1 [00:00<?, ?it/s]

['Как пояснили в государственной Федерации, Организации, которая была одобрена Советом руководителей таможенных службы только уставены, которая была я подчинена Советом, подчинения всех стран, сНГ за исключены Армении.']

In [28]:
train_df.loc[1]["input_text"]

'как пояснили в государственном таможенном комитете (гтк) форма декларации которая была одобрена советом руководителей таможенных служб государств-участников содружества является единой на территории всех стран снг за исключением армении.'

In [27]:
train_df.loc[1]["target_text"]

'Как пояснили в Государственном таможенном комитете (ГТК), форма декларации, которая была одобрена Советом руководителей таможенных служб государств-участников Содружества, является единой на территории всех стран СНГ, за исключением Армении.'